In [66]:
import numpy as np
import math
from prettytable import PrettyTable

Варіант 22

In [67]:
# Теоретичний розподіл: РОЗПОДІЛ ПУАССОНА

data = np.array([12, 15, 7, 19, 15, 27, 22, 4,
                 11, 19, 15, 7, 1, 12, 22, 4,
                 11, 15, 19, 12, 7, 27, 15, 19,
                 11, 22, 12, 19, 15, 12, 15, 31])
y = 0.999
sigma = 7
a = 0.05

In [68]:
var_arr = np.sort(data)
values, counts = np.unique(var_arr, return_counts=True)
n = np.sum(counts)

table = PrettyTable()
table.field_names = ["Унікальне значення", "Частота"]
for i in range(np.size(values)):
    table.add_row([values[i], counts[i]], divider=True)
table

Унікальне значення,Частота
1,1
4,2
7,3
11,3
12,5
15,7
19,5
22,3
27,2
31,1


In [69]:
print(f'n = {n}')

n = 32


Знайдіть точкові незміщені оцінки середнього значення та середнього квадратичного відхилення

In [70]:
x_b = round(np.sum(values * counts) / n, 6)
print(x_b)

14.8125


In [71]:
D_b = round(np.sum(pow(values, 2) * counts) / n, 6) - pow(x_b, 2)
print(D_b)

46.65234375


In [72]:
S2 = D_b*n/(n-1)
print(S2)

48.15725806451613


In [86]:
sigma_b = round(np.sqrt(D_b), 6)
print(sigma_b)

6.830252


In [88]:
S = round(np.sqrt(S2), 6)
print(S)

6.939543


Припускаючи нормальний розподіл випадкової величини, знайдіть з надійністю у довірчий інтервал для математичного сподівання при відомому σ 

х взято з таблички

In [74]:
# F_x = y/2 = 0.4995
x_res = 3.4

top_limit = x_b - x_res*sigma/np.sqrt(n)
bottom_limit = x_b + x_res*sigma/np.sqrt(n)

print(x_res)
print(f'{round(top_limit, 6)} < M(x) < {round(bottom_limit, 6)}')

3.4
10.605215 < M(x) < 19.019785


х через norm.ppf()

In [75]:
from scipy.stats import norm

In [76]:
x_res1 = norm.ppf(y)

top_limit = x_b - x_res1*sigma/np.sqrt(n)
bottom_limit = x_b + x_res1*sigma/np.sqrt(n)

print(round(x_res1, 2))
print(f'{round(top_limit, 6)} < M(x) < {round(bottom_limit, 6)}')

3.09
10.988533 < M(x) < 18.636467


х через erf()
https://stackoverflow.com/questions/56016484/how-to-calculate-laplace-function-in-python-3

In [77]:
from scipy.special import erf

In [78]:
F_x = lambda x: erf(x/2**0.5)/2
x_res2 = F_x(y)

top_limit = x_b - x_res2*sigma/np.sqrt(n)
bottom_limit = x_b + x_res2*sigma/np.sqrt(n)

print(round(x_res2, 2))
print(f'{round(top_limit, 6)} < M(x) < {round(bottom_limit, 6)}')

0.34
14.390407 < M(x) < 15.234593


Перевірте за допомогою критерію Пірсона при рівні значущості α гіпотезу про відповідний теоретичний розподіл заданої випадкової величини

In [79]:
from scipy.stats import chi2

In [80]:
def poisson(x_b, m):
    return (np.power(x_b, m) * np.exp(-m)) / math.factorial(m)

In [81]:
p = np.array([])
for x in values:
    p = np.append(p, poisson(x_b, x))

p = np.append(p, 1 - np.sum(p))
n_p = np.array([n * p_i for p_i in p])

pearson_values = np.array([np.power(n_i - np_i,2) / np_i for n_i, np_i in zip(counts, n_p)])
pearson_observed =np.sum(pearson_values)

In [89]:
table = PrettyTable()
table.field_names = ["x_i", "n_i", "p_i", "n*_i","(n_i-n_i*)^2", "(n_i-n_i*)^2 / n_i*"]
midpoints = np.array([np.power(n_i - np_i, 2) for n_i, np_i in zip(counts, n_p)])

for i in range(len(values)):
    table.add_row([f"{values[i]}",
                   counts[i],
                   f"{p[i]:.4f}",
                   f"{n_p[i]:.4f}",
                   f"{midpoints[i]:.4f}",
                   f"{pearson_values[i]:.4f}"])

table.add_row(["Sum", np.sum(counts), np.sum(p), np.sum(n_p), "-", f"χ^2 спост = {np.sum(pearson_values):.4f}"])
table

x_i,n_i,p_i,n*_i,(n_i-n_i*)^2,(n_i-n_i*)^2 / n_i*
1,1,5.4492,174.3749,30058.8404,172.3806
4,2,36.7387,1175.6398,1377430.2670,1171.6432
7,3,28.3078,905.8507,815139.3404,899.8606
11,3,3.1515,100.8479,9574.2159,94.9372
12,5,1.4311,45.7952,1664.2452,36.3411
15,7,0.0848,2.7143,18.3672,6.7668
19,5,0.0008,0.0257,24.7434,961.7452
22,3,0.0000,0.0005,8.9973,19970.2397
27,2,0.0000,0.0000,4.0000,17901025.3573
31,1,0.0000,0.0000,1.0000,3832846839.8033


In [83]:
k = len(values) - 1 - 1
print(k)

8


In [91]:
critical_value = chi2.ppf(1 - a, k)
print(f"Критичне значення χ^2 для α = {a} та k = {k}: {critical_value:.4f}")

Критичне значення χ^2 для α = 0.05 та k = 8: 15.5073


In [90]:
if pearson_observed < critical_value:
    print("Оскільки χ^2спост < χ^2кр, гіпотезу про розподіл Пуассона не відхиляємо")
else:
    print("Оскільки χ^2спост > χ^2кр, гіпотезу про розподіл Пуассона відхиляємо")

Оскільки χ^2спост > χ^2кр, гіпотезу про розподіл Пуассона відхиляємо
